# Classification Based Machine Learning Algorithm

**Supervised learning**, in which the data comes with additional attributes that we want to predict. This problem can be either:

* **Classification**: samples belong to two or more *classes* and we want to learn from already labeled data how to predict the class of unlabeled data. An example of classification problem would be the handwritten digit recognition example, in which the aim is to assign each input vector to one of a finite number of discrete categories. Another way to think of classification is as a discrete (as opposed to continuous) form of supervised learning where one has a limited number of categories and for each of the n samples provided, one is to try to label them with the correct category or class.


* **Regression**: if the desired output consists of one or more *continuous variables*, then the task is called regression. An example of a regression problem would be the prediction of the length of a salmon as a function of its age and weight.

MNIST dataset - a set of 70,000 small images of digits handwritten. You can read more via [The MNIST Database](http://yann.lecun.com/exdb/mnist/)

***

## Downloading the MNIST dataset

In [ ]:
import numpy as np

In [ ]:
from sklearn.datasets import fetch_mldata
# I wanna store the MNIST dataset in my desired directory, change it if you want.
mnist = fetch_mldata('MNIST original', data_home='C:\\MNIST')

In [ ]:
# what is MNIST?!
mnist

In [ ]:
# how many picures we have?!
len(mnist['data'])

# Visualisation

In [ ]:
# Oh god! spliting again!
X, y = mnist['data'], mnist['target']

In [ ]:
X

In [ ]:
y

In [ ]:
# a sample number call
X[69999]

In [ ]:
# it's a cute little nine!
y[69999]

In [ ]:
# numbers are obvoiusly arrays.
X.shape

In [ ]:
# and their label is a solid integer.
y.shape

In [ ]:
# importing what we need
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ploting a sample digit
sample_digit = X[1000]
sample_digit_image = sample_digit.reshape(28, 28)
plt.imshow(sample_digit_image);

In [ ]:
# what number it is?!
y[1000]

### Exercise: Locating the number 4 and plot the image

In [ ]:
type(y)

In [ ]:
# return true in values where actually are four xD
y == 4

In [ ]:
# where is the fours?! (using dear numpy)
np.where(y==4)

In [ ]:
# first 'four' occures in 24755'th digit. 
y[24754]

In [ ]:
# showing the first four of our dataset
# sry for variable 
first_four_of_dataset = X[24754]
first_four_of_dataset_image = first_four_of_dataset.reshape(28, 28)
plt.imshow(first_four_of_dataset_image);

***

# Splitting the train and test sets

In [ ]:
num_split = 60000
# and taking the remaining 10000 for test 
X_train, X_test, y_train, y_test = X[:num_split], X[num_split:], y[:num_split], y[num_split:]

**Tips**: Typically we shuffle the training set. This ensures the training set is randomised and your data distribution is consistent. However, shuffling is a bad idea for time series data.

# Shuffling the dataset

[Alternative Method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) is much better and simple.

In [ ]:
import numpy as np

In [ ]:
shuffle_index = np.random.permutation(num_split)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

## Training a Binary Classifier

To simplify our problem, we will make this an exercise of "zero" or "non-zero", making it a two-class problem.

We need to first convert our target to 0 or non zero.

In [ ]:
# creating a new variable, which where the labels are zero
# for training set
y_train_0 = (y_train == 0)

In [ ]:
y_train_0

In [ ]:
# creating a new variable, which where the labels are zero
# for testing set
y_test_0 = (y_test == 0)

In [ ]:
y_test_0

At this point we can pick any classifier and train it. This is the iterative part of choosing and testing all the classifiers and tuning the hyper parameters

***

# SGDClassifier

# Training

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(random_state = 0, max_iter=5, tol=None)
clf.fit(X_train, y_train_0)

# Prediction

In [ ]:
clf.predict(X[1000].reshape(1, -1))

In [ ]:
print(y_train_0[1000])

so, Fuck !

***

# Performance Measures

# Measuring Accuracy Using Cross-Validation

## StratifiedKFold

Let's try with the `StratifiedKFold` stratified sampling to create multiple folds. At each iteration, the classifier was cloned and trained using the training folds and makes predictions on the test fold. 

StratifiedKFold utilised the Stratified sampling concept, but with this features and modifications:

* The population is divided into homogeneous subgroups called strata
* The right number of instances is sampled from each stratum 
* To guarantee that the test set is representative of the population

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
# instantiating the SGD Classifier
clf = SGDClassifier(random_state=0,max_iter=5, tol=None)

In [ ]:
# setting the number of splits for our folds
# and using the random_state parameter just for assuring that the values in test set are both zero and non-zero
# awareing that about 10% of MNIST is zero and the rest is non-zero
skfolds = StratifiedKFold(n_splits=3, random_state=100)

In [ ]:
# for our train and test queries in k-fold splits...
# we gonna loop through this for loop in amount of n_splits (e.g 3 times)
for train_index, test_index in skfolds.split(X_train, y_train_0):
    # clone the classifier instance     
    clone_clf = clone(clf)
    
    X_train_fold = X_train[train_index]
    y_train_folds = (y_train_0[train_index])
    
    X_test_fold = X_train[test_index]
    y_test_fold = (y_train_0[test_index])
    
    # fit them all !
    clone_clf.fit(X_train_fold, y_train_folds)
    # predicting some test data     
    y_pred = clone_clf.predict(X_test_fold)
    # achieving the number of correct predictions...
    n_correct = sum(y_pred == y_test_fold)
    print("{0:.4f}".format(n_correct / len(y_pred)))

#### `cross_val_score` using K-fold Cross-Validation

actually functionality of code below is the same as the code we implemented above. in other words, stuff we did above was implemented already.

K-fold cross-validation splits the training set into K-folds and then make predictions and evaluate them on each fold using a model trained on the remaning folds.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(clf, X_train, y_train_0, cv=3, scoring='accuracy')

#### Exercise:

What if you would like to perform 10-fold CV test? How would you do that

In [ ]:
cross_val_score(clf, X_train, y_train_0, cv=10, scoring='accuracy')

***

## Danger of Blindly Applying Evaluator As a Performance Measure

Let's check against a dumb classifier

In [ ]:
1 - sum(y_train_0) / len(y_train_0)

A simple check shows that 90.1% of the images are not zero. Any time you guess the image is not zero, you will be right 90.13% of the time. 

Bare this in mind when you are dealing with **skewed datasets**. Because of this, accuracy is generally not the preferred performance measure for classifiers.

# Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
# make some predictions, using cross validation
y_train_pred = cross_val_predict(clf, X_train, y_train_0, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# building the confusion matrix using the actual target data and the data we predicted
confusion_matrix(y_train_0, y_train_pred)

####  actually numbers can be differ in every execution...

Each row: actual class

Each column: predicted class

First row: Non-zero images, the negative class:
* 53360 were correctly classified as non-zeros. **True negatives**. 
* Remaining 717 were wrongly classified as 0s. **False positive**


Second row: The images of zeros, the positive class:
* 395 were incorrectly classified as 0s. **False negatives**
* 5528 were correctly classified as 0s. **True positives**


<img src="img\confusion matrix.jpg">

# Precision

**Precision** measures the accuracy of positive predictions. Also called the `precision` of the classifier

$$\textrm{precision} = \frac{\textrm{True Positives}}{\textrm{True Positives} + \textrm{False Positives}}$$

<img src="img\precision.jpg">

In [ ]:
from sklearn.metrics import precision_score, recall_score

Note the result here may vary from the video as the results from the confusion matrix are different each time you run it.

In [ ]:
precision_score(y_train_0, y_train_pred) # 5618 / (574 + 5618)

In [ ]:
# actually numbers can be differ in every execution...
5618 / (574 + 5618)

## Recall

`Precision` is typically used with `recall` (`Sensitivity` or `True Positive Rate`). The ratio of positive instances that are correctly detected by the classifier.

$$\textrm{recall} = \frac{\textrm{True Positives}}{\textrm{True Positives} + \textrm{False Negatives}}$$

<img src="img\recall.jpg">

Note the result here may vary from the video as the results from the confusion matrix are different each time you run it.

In [ ]:
recall_score(y_train_0, y_train_pred) # 5618 / (305 + 5618)

In [ ]:
5618 / (305 + 5618)

## F1 Score

$F_1$ score is the harmonic mean of precision and recall. Regular mean gives equal weight to all values. Harmonic mean gives more weight to low values.


$$F_1=\frac{2}{\frac{1}{\textrm{precision}}+\frac{1}{\textrm{recall}}}=2\times \frac{\textrm{precision}\times \textrm{recall}}{\textrm{precision}+ \textrm{recall}}=\frac{TP}{TP+\frac{FN+FP}{2}}$$

The $F_1$ score favours classifiers that have similar precision and recall.


In [ ]:
from sklearn.metrics import f1_score

Note the result here may vary from the video as the results from the confusion matrix are different each time you run it.

In [ ]:
f1_score(y_train_0, y_train_pred)

***

# Precision / Recall Tradeoff
## Increasing precision make recall reduce and vice versa

<img src="img\precision-recall.png">

Our classifier is designed to pick up zeros.

12 observations

***

**Central Arrow**

Suppose the decision threshold is positioned at the central arrow: 
* We get 4 true positives (We have 4 zeros to the right of the central arrow)
* 1 false positive which is actually seven.

At this threshold, the **precision accuracy** is $\frac{4}{5}=80\%$

However, out of the 6 zeros, the classifier only picked up 4. The **recall accuracy** is $\frac{4}{6}=67\%$

***

**Right Arrow**

* We get 3 true positives
* 0 false positive

At this threshold, the **precision accuracy** is $\frac{3}{3}=100\%$
However, out of the 6 zeros, the classifier only picked up 3. The **recall accuracy** is $\frac{3}{6}=50\%$

***

**Left Arrow**

* We get 6 true positives
* 2 false positive

At this threshold, the **precision accuracy** is $\frac{6}{8}=75\%$
Out of the 6 zeros, the classifier picked up all 6. The **recall accuracy** is $\frac{6}{6}=100\%$

***




In [ ]:
clf = SGDClassifier(random_state=0, max_iter=5, tol=None)
clf.fit(X_train, y_train_0)

In [ ]:
y[1000]

In [ ]:
y_scores = clf.decision_function(X[1000].reshape(1, -1))
y_scores

In [ ]:
threshold = 0

In [ ]:
y_some_digits_pred = (y_scores > threshold)

In [ ]:
y_some_digits_pred

In [ ]:
threshold = 40000
y_some_digits_pred = (y_scores > threshold)
y_some_digits_pred

In [ ]:
y_scores = cross_val_predict(clf, X_train, y_train_0, cv=3, method='decision_function')

In [ ]:
plt.figure(figsize=(12,8)); plt.hist(y_scores, bins=100);

With the decision scores, we can compute precision and recall for all possible thresholds using the `precision_recall_curve()` function:

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train_0, y_scores)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g--", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([-0.5,1.5])    

In [ ]:
plt.figure(figsize=(12,8)); 
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

With this chart, you can select the threshold value that gives you the best precision/recall tradeoff for your task.

Some tasks may call for higher precision (accuracy of positive predictions). Like designing a classifier that picks up adult contents to protect kids. This will require the classifier to set a high bar to allow any contents to be consumed by children.

Some tasks may call for higher recall (ratio of positive instances that are correctly detected by the classifier). Such as detecting shoplifters/intruders on surveillance images - Anything that remotely resemble "positive" instances to be picked up.

***

One can also plot precisions against recalls to assist with the threshold selection

In [ ]:
plt.figure(figsize=(12,8)); 
plt.plot(precisions, recalls);
plt.xlabel('recalls');
plt.ylabel('precisions');
plt.title('PR Curve: precisions/recalls tradeoff');

# Setting High Precisions

Let's aim for 90% precisions.

In [ ]:
len(precisions)

In [ ]:
len(thresholds)

In [ ]:
plt.figure(figsize=(12,8)); 
plt.plot(thresholds, precisions[1:]);

In [ ]:
idx = len(precisions[precisions < 0.9])

In [ ]:
thresholds[idx]

In [ ]:
y_train_pred_90 = (y_scores > 21454)

In [ ]:
precision_score(y_train_0, y_train_pred_90)

In [ ]:
recall_score(y_train_0, y_train_pred_90)

# Setting High Precisions

Let's aim for 99% precisions.

In [ ]:
idx = len(precisions[precisions < 0.99])

This is the same as the line above

In [ ]:
thresholds[idx]

In [ ]:
y_train_pred_90 = (y_scores > thresholds[idx])

In [ ]:
precision_score(y_train_0, y_train_pred_90)

In [ ]:
recall_score(y_train_0, y_train_pred_90)

#### Exercise

High Recall Score. Recall score > 0.9

In [ ]:
idx = len(recalls[recalls > 0.9])

In [ ]:
thresholds[idx]

In [ ]:
y_train_pred_90 = (y_scores > thresholds[idx])

In [ ]:
precision_score(y_train_0, y_train_pred_90)

In [ ]:
recall_score(y_train_0, y_train_pred_90)

***

## The Receiver Operating Characteristics (ROC) Curve

Instead of plotting precision versus recall, the ROC curve plots the `true positive rate` (another name for recall) against the `false positive rate`. The `false positive rate` (FPR) is the ratio of negative instances that are incorrectly classified as positive. It is equal to one minus the `true negative rate`, which is the ratio of negative instances that are correctly classified as negative.

The TNR is also called `specificity`. Hence the ROC curve plots `sensitivity` (recall) versus `1 - specificity`.

<img src="img\tnr_and_fpr.png">

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train_0, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1], [0,1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')

In [ ]:
plt.figure(figsize=(12,8)); 
plot_roc_curve(fpr, tpr)
plt.show();

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_train_0, y_scores)

Use PR curve whenever the **positive class is rare** or when you care more about the false positives than the false negatives

Use ROC curve whenever the **negative class is rare** or when you care more about the false negatives than the false positives


In the example above, the ROC curve seemed to suggest that the classifier is good. However, when you look at the PR curve, you can see that there are room for improvement.

# Model Comparison

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
f_clf = RandomForestClassifier(random_state=0)

In [ ]:
y_probas_forest = cross_val_predict(f_clf, X_train, y_train_0,
                                   cv=3, method='predict_proba')

In [ ]:
y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, threshold_forest = roc_curve(y_train_0, y_scores_forest)

In [ ]:
plt.figure(figsize=(12,8)); 
plt.plot(fpr, tpr, "b:", label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right")
plt.show();

In [ ]:
roc_auc_score(y_train_0, y_scores_forest)

In [ ]:
f_clf.fit(X_train, y_train_0)

In [ ]:
y_train_rf = cross_val_predict(f_clf, X_train, y_train_0, cv=3)

In [ ]:
precision_score(y_train_0, y_train_rf) 

In [ ]:
recall_score(y_train_0, y_train_rf) 

In [ ]:
confusion_matrix(y_train_0, y_train_rf)

***